In [ ]:
import os
import sys
sys.path.append('..')

### Load checkpoint

In [ ]:
os.makedirs('models', exist_ok=True)

In [ ]:
!wget https://huggingface.co/booydar/rmt-gpt2-babilong-qa1/resolve/main/pytorch_model.bin -P models/
# !wget https://huggingface.co/booydar/rmt-gpt2-babilong-qa2/resolve/main/pytorch_model.bin -P models/
# !wget https://huggingface.co/booydar/rmt-gpt2-babilong-qa3/resolve/main/pytorch_model.bin -P models/

--2024-08-09 12:00:17--  https://huggingface.co/booydar/rmt-gpt2-babilong-qa1/resolve/main/pytorch_model.bin
Resolving huggingface.co (huggingface.co)... 18.239.50.103, 18.239.50.16, 18.239.50.49, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.103|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/2b/a6/2ba65b402b075ff6c41f583ffeea745cccc04e1aa48ec0a7895dd3ec95947c08/a2de2b7590e193a6c6d48cfe5b8b684643307fbe0655726986e456f7da17370c?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1723464017&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMzQ2NDAxN319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzJiL2E2LzJiYTY1YjQwMmIwNzVmZjZjNDFmNTgzZmZlZWE3NDVjY2NjMDRlMWFhNDhlYzBhNzg5NWRkM2VjOTU5NDdjMDgvYTJkZTJiNzU5MGUxOTNhNm

In [ ]:
!git clone https://github.com/booydar/recurrent-memory-transformer/ -b babilong-release github
sys.path.append('./github')

Cloning into 'github'...
remote: Enumerating objects: 5549, done.
remote: Counting objects: 100% (1826/1826), done.
remote: Compressing objects: 100% (542/542), done.
remote: Total 5549 (delta 1345), reused 1686 (delta 1277), pack-reused 3723
Receiving objects: 100% (5549/5549), 79.74 MiB | 6.76 MiB/s, done.
Resolving deltas: 100% (3775/3775), done.


### Create RMT model

In [ ]:
!pip install datasets

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from modeling_rmt.language_modeling import RecurrentWrapper, MemoryCell
from datasets import load_dataset

In [ ]:
device = 'cpu'
tokenizer = AutoTokenizer.from_pretrained('gpt2')

model = AutoModelForCausalLM.from_pretrained('gpt2')
cell = MemoryCell(model, num_mem_tokens=16)

model = RecurrentWrapper(cell,
                            segment_size=512,
                            max_n_segments=32,
).to(device)


model_cpt = "./models/pytorch_model.bin"
cpt = torch.load(model_cpt, map_location=device)
model.load_state_dict(cpt, strict=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

<All keys matched successfully>

### Prepare dataset

In [ ]:
task = 'qa1'
seq_len = '0k'
ds = load_dataset('RMT-team/babilong', seq_len)[task]

In [ ]:
sample_num = 0
sample = ds[sample_num]

In [ ]:
sample

{'input': 'John travelled to the hallway. Mary journeyed to the bathroom. Daniel went back to the bathroom. John moved to the bedroom.',
 'question': 'Where is Mary? ',
 'target': 'bathroom'}

### Get prediction

In [ ]:
input_text = f"{sample['input']} {sample['question']}"
model_input = tokenizer.encode_plus(input_text, return_tensors='pt').to(device)
out = model.generate(**model_input, max_new_tokens=20)
prediction = tokenizer.decode(out[0][:-1], add_special_tokens=False)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:

print(f"Task: {input_text[:100]}\nPrediction: {prediction}\nAnswer: {sample['target']}\n{'Correct!' if sample['target'] == prediction else 'Wrong'}")

Task: John travelled to the hallway. Mary journeyed to the bathroom. Daniel went back to the bathroom. Joh
Prediction: bathroom
Answer: bathroom
Correct!
